<a href="https://colab.research.google.com/github/saiku122/AIJobcolle/blob/master/LatstTask_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/saiku122/AIJobcolle.git

Cloning into 'AIJobcolle'...
remote: Enumerating objects: 383, done.
remote: Counting objects: 100% (383/383), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 383 (delta 160), reused 201 (delta 65), pack-reused 0
Receiving objects: 100% (383/383), 11.88 MiB | 14.02 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [ ]:
cd /content/AIJobcolle/MachineLearning/finaltask/

/content/AIJobcolle/MachineLearning/finaltask


★★最終課題★★</br>
〇入力データ</br>
・モデル用：final_hr_analysis_train.csv</br>
・スコア用：final_hr_analysis_test.csv</br>
</br>
〇出力データ</br>
・スコア付データ：</br> 
・モデル：</br>


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import joblib
from sklearn.metrics import accuracy_score
import lightgbm as lgb

In [ ]:
#インプットファイル
file_model_name = 'final_hr_analysis_train.csv'
file_test_name = 'final_hr_analysis_test.csv'

In [ ]:
my_dtype = {'mode_category':object}

In [ ]:
#  モデル用データ読み込み
df = pd.read_csv('./input/'+ file_model_name,
                 header=0,
                 dtype=my_dtype)
ID = df.iloc[:,0] #1列目取得
y=df.iloc[:,1]    #2列目の正解データを取得
X=df.iloc[:,2:]   #3列目以降取得
df.dtypes

customerid                     int64
tgt                            int64
trips                          int64
amount_of_yen                float64
quantity                       int64
purchase_period                int64
purchase_recency               int64
purchase_amount_per_trips    float64
regularity                   float64
trips_cancellation             int64
trips_cat85099                 int64
trips_cat85123                 int64
trips_cat22423                 int64
trips_cat47566                 int64
trips_cat84879                 int64
trips_cat20725                 int64
trips_cat22720                 int64
trips_catpost                  int64
trips_cat23203                 int64
trips_cat22383                 int64
mode_category                 object
dtype: object

In [ ]:
# one-hot encoding処理
ohe_cols = ['mode_category'] #!!!変更箇所
X_ohe = pd.get_dummies(X, dummy_na=True, columns=ohe_cols)
#display(X_ohe)
X_ohe = X_ohe.dropna(axis=1,how='all')#axis=1　列
#display(X_ohe)
X_ohe_columns = X_ohe.columns.values #モデルデータの列確定

In [ ]:
#欠損値の置き換え
imp=SimpleImputer() #平均値で欠損値を補完
imp.fit(X_ohe)
X_ohe = pd.DataFrame(imp.transform(X_ohe),columns=X_ohe_columns) # モデルデータの列を平均値補完適用させ変数を更新

In [ ]:
#特徴量抽出
selector = RFECV(estimator=RandomForestClassifier(n_estimators=100,random_state=0), step=0.05) #特徴量をRFE(主成分分析)を用いて次元削減する。(モデルはランダムフォレスト)
selector.fit(X_ohe, y) #RFEした結果を保存
X_ohe_selected = selector.transform(X_ohe) #X_oheに反映
X_ohe_selected = pd.DataFrame(X_ohe_selected, columns=X_ohe_columns[selector.support_]) #削減実行後のデータを再構成

**テストデータの読み込み**<br>
モデル用データのエンコード結果に列名をあわせる

In [ ]:
#  テスト用データ読み込み
df_s = pd.read_csv('./input/'+ file_test_name,
                 header=0,
                 dtype=my_dtype)
ID_s = df_s.iloc[:,[0]] #1列目取得
y_s=df_s.iloc[:,1]    #2列目の正解データを取得
X_s=df_s.iloc[:,2:]   #3列目以降取得
df.dtypes

customerid                     int64
tgt                            int64
trips                          int64
amount_of_yen                float64
quantity                       int64
purchase_period                int64
purchase_recency               int64
purchase_amount_per_trips    float64
regularity                   float64
trips_cancellation             int64
trips_cat85099                 int64
trips_cat85123                 int64
trips_cat22423                 int64
trips_cat47566                 int64
trips_cat84879                 int64
trips_cat20725                 int64
trips_cat22720                 int64
trips_catpost                  int64
trips_cat23203                 int64
trips_cat22383                 int64
mode_category                 object
dtype: object

In [ ]:
# one-hot encoding処理
X_ohe_s = pd.get_dummies(X_s, dummy_na=True, columns=ohe_cols)
#display(X_ohe)
X_ohe_s = X_ohe_s.dropna(axis=1,how='all')#axis=1　列
#display(X_ohe)
X_ohe_columns_s = X_ohe_s.columns.values #テストデータの仮確定

In [ ]:
#列の整合性チェック
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

diff1 = cols_model - cols_score
print('モデルのみに存在する項目: %s' % diff1)

diff2 = cols_score - cols_model
print('スコアのみに存在する項目: %s' % diff2)

モデルのみに存在する項目: {'mode_category_84832', 'mode_category_22461', 'mode_category_22470', 'mode_category_21531', 'mode_category_85131', 'mode_category_21908', 'mode_category_22655', 'mode_category_22148', 'mode_category_22722', 'mode_category_22752', 'mode_category_21626', 'mode_category_22690', 'mode_category_21116', 'mode_category_85202', 'mode_category_22926', 'mode_category_21669', 'mode_category_22789', 'mode_category_20971', 'mode_category_23092', 'mode_category_47503', 'mode_category_22087', 'mode_category_37450', 'mode_category_21136', 'mode_category_22437', 'mode_category_48188', 'mode_category_22059', 'mode_category_21843', 'mode_category_82001', 'mode_category_85059', 'mode_category_35004', 'mode_category_21700', 'mode_category_21609', 'mode_category_85040', 'mode_category_84849', 'mode_category_22303', 'mode_category_84944', 'mode_category_22469', 'mode_category_22664', 'mode_category_84992', 'mode_category_21216', 'mode_category_85206', 'mode_category_85194', 'mode_category_7504

In [ ]:
#列名一致
df_cols_m = pd.DataFrame(None,
                         columns=X_ohe_columns,
                         dtype=float)
X_ohe_s2 = pd.concat([df_cols_m, X_ohe_s])
#display(X_ohe_s2.head(3))

set_Xm = set(X_ohe.columns.values)
set_Xs = set(X_ohe_s.columns.values)

#スコアにのみ存在する項目の削除
X_ohe_s3 = X_ohe_s2.drop(list(set_Xs-set_Xm),axis=1)

#print(X_ohe_s3.shape)
#display(X_ohe_s3.head(3))

#list(set_Xs-set_Xm)

In [ ]:
#スコアにのみ存在する項目の削除
X_ohe_s3.loc[:,list(set_Xm-set_Xs)] = X_ohe_s3.loc[:,list(set_Xm-set_Xs)].fillna(0,axis=1)
X_ohe_s3.head(3)

,trips,amount_of_yen,quantity,purchase_period,purchase_recency,purchase_amount_per_trips,regularity,trips_cancellation,trips_cat85099,trips_cat85123,trips_cat22423,trips_cat47566,trips_cat84879,trips_cat20725,trips_cat22720,trips_catpost,trips_cat23203,trips_cat22383,mode_category_15044,mode_category_15056,mode_category_20718,mode_category_20725,mode_category_20727,mode_category_20728,mode_category_20971,mode_category_21034,mode_category_21084,mode_category_21116,mode_category_21136,mode_category_21137,mode_category_21181,mode_category_21216,mode_category_21313,mode_category_21314,mode_category_21326,mode_category_21340,mode_category_21417,mode_category_21466,mode_category_21495,mode_category_21519,...,mode_category_85036,mode_category_85039,mode_category_85040,mode_category_85049,mode_category_85059,mode_category_85066,mode_category_85071,mode_category_85099,mode_category_85114,mode_category_85116,mode_category_85118,mode_category_85123,mode_category_85131,mode_category_85132,mode_category_85141,mode_category_85152,mode_category_85159,mode_category_85169,mode_category_85170,mode_category_85175,mode_category_85178,mode_category_85180,mode_category_85183,mode_category_85186,mode_category_85194,mode_category_85202,mode_category_85206,mode_category_85212,mode_category_85230,mode_category_85231,mode_category_85232,mode_category_90125,mode_category_90200,mode_category_90201,mode_category_90214,mode_category_C2,mode_category_D,mode_category_M,mode_category_POST,mode_category_nan
0,5.0,2080.34,1018.0,153.0,2.0,416.068,30.6,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4.0,661.64,293.0,106.0,42.0,165.410,26.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5.0,1265.48,679.0,147.0,28.0,253.096,29.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#ソート順の統一
X_ohe_s3 = X_ohe_s3.reindex(X_ohe.columns.values,axis=1)
X_ohe_s3.head(3)

,trips,amount_of_yen,quantity,purchase_period,purchase_recency,purchase_amount_per_trips,regularity,trips_cancellation,trips_cat85099,trips_cat85123,trips_cat22423,trips_cat47566,trips_cat84879,trips_cat20725,trips_cat22720,trips_catpost,trips_cat23203,trips_cat22383,mode_category_15044,mode_category_15056,mode_category_20718,mode_category_20725,mode_category_20727,mode_category_20728,mode_category_20971,mode_category_21034,mode_category_21084,mode_category_21116,mode_category_21136,mode_category_21137,mode_category_21181,mode_category_21216,mode_category_21313,mode_category_21314,mode_category_21326,mode_category_21340,mode_category_21417,mode_category_21466,mode_category_21495,mode_category_21519,...,mode_category_85036,mode_category_85039,mode_category_85040,mode_category_85049,mode_category_85059,mode_category_85066,mode_category_85071,mode_category_85099,mode_category_85114,mode_category_85116,mode_category_85118,mode_category_85123,mode_category_85131,mode_category_85132,mode_category_85141,mode_category_85152,mode_category_85159,mode_category_85169,mode_category_85170,mode_category_85175,mode_category_85178,mode_category_85180,mode_category_85183,mode_category_85186,mode_category_85194,mode_category_85202,mode_category_85206,mode_category_85212,mode_category_85230,mode_category_85231,mode_category_85232,mode_category_90125,mode_category_90200,mode_category_90201,mode_category_90214,mode_category_C2,mode_category_D,mode_category_M,mode_category_POST,mode_category_nan
0,5.0,2080.34,1018.0,153.0,2.0,416.068,30.6,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4.0,661.64,293.0,106.0,42.0,165.410,26.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5.0,1265.48,679.0,147.0,28.0,253.096,29.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#欠損値処理
X_ohe_s4 = pd.DataFrame(imp.transform(X_ohe_s3),columns=X_ohe_columns)

In [ ]:
#RFE適用後の列名のみテストデータから抽出
X_ohe_selected_s=X_ohe_s4.loc[:, X_ohe_columns[selector.support_]]

In [ ]:
##テスト実行（最後本当にID_sとくっつけていのか？）
X_ohe_s4

,trips,amount_of_yen,quantity,purchase_period,purchase_recency,purchase_amount_per_trips,regularity,trips_cancellation,trips_cat85099,trips_cat85123,trips_cat22423,trips_cat47566,trips_cat84879,trips_cat20725,trips_cat22720,trips_catpost,trips_cat23203,trips_cat22383,mode_category_15044,mode_category_15056,mode_category_20718,mode_category_20725,mode_category_20727,mode_category_20728,mode_category_20971,mode_category_21034,mode_category_21084,mode_category_21116,mode_category_21136,mode_category_21137,mode_category_21181,mode_category_21216,mode_category_21313,mode_category_21314,mode_category_21326,mode_category_21340,mode_category_21417,mode_category_21466,mode_category_21495,mode_category_21519,...,mode_category_85036,mode_category_85039,mode_category_85040,mode_category_85049,mode_category_85059,mode_category_85066,mode_category_85071,mode_category_85099,mode_category_85114,mode_category_85116,mode_category_85118,mode_category_85123,mode_category_85131,mode_category_85132,mode_category_85141,mode_category_85152,mode_category_85159,mode_category_85169,mode_category_85170,mode_category_85175,mode_category_85178,mode_category_85180,mode_category_85183,mode_category_85186,mode_category_85194,mode_category_85202,mode_category_85206,mode_category_85212,mode_category_85230,mode_category_85231,mode_category_85232,mode_category_90125,mode_category_90200,mode_category_90201,mode_category_90214,mode_category_C2,mode_category_D,mode_category_M,mode_category_POST,mode_category_nan
0,5.0,2080.34,1018.0,153.0,2.0,416.068000,30.600000,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4.0,661.64,293.0,106.0,42.0,165.410000,26.500000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5.0,1265.48,679.0,147.0,28.0,253.096000,29.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7.0,11056.93,5543.0,103.0,8.0,1579.561429,14.714286,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13.0,63661.49,43090.0,152.0,29.0,4897.037692,11.692308,0.0,4.0,0.0,2.0,2.0,0.0,3.0,4.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,4.0,783.02,501.0,72.0,43.0,195.755000,18.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
803,5.0,1150.47,579.0,92.0,44.0,230.094000,18.400000,0.0,1.0,1.0,1.0,0.0,0.0,1.0

**モデリング**

In [ ]:
#アルゴリズム定義
model_name = 'GBC_001'
'''
clf = Pipeline([('scl',StandardScaler()), 
                ('est',GradientBoostingClassifier(random_state=1))
                ])
'''
# set pipelines for different algorithms
pipelines = {
#    'knn':
#       Pipeline([('scl',StandardScaler()),
#                  ('est',KNeighborsClassifier())]),
#    'logistic':
#        Pipeline([('scl',StandardScaler()),
#                  ('est',LogisticRegression(random_state=1))]),
#    'rsvc':
#        Pipeline([('scl',StandardScaler()),
#                  ('est',SVC(C=1.0, kernel='rbf', class_weight='balanced', random_state=1))]),
#    'lsvc':
#        Pipeline([('scl',StandardScaler()),
#                  ('est',LinearSVC(C=1.0, class_weight='balanced', random_state=1))]),
    'tree':
        Pipeline([('scl',StandardScaler()),
                  ('est',DecisionTreeClassifier(random_state=1))]),
    'rf':
        Pipeline([('scl',StandardScaler()),
                  ('est',RandomForestClassifier(random_state=1))]),
    'gb':
        Pipeline([('scl',StandardScaler()),
                  ('est',GradientBoostingClassifier(random_state=1))]),
    'lgb':
        Pipeline([('scl',StandardScaler()),
                  ('lgb',lgb.LGBMClassifier())]),
    'mlp': #ニューラルネットワーク
        Pipeline([('scl',StandardScaler()),
                  ('est',MLPClassifier(hidden_layer_sizes=(3,3),
                                       max_iter=1000,
                                       random_state=1))])
}

In [ ]:
#clf.fit(X_ohe_selected, y.values.ravel())#Xを標準化、勾配ブースティングを適用)
scores = {}
for pipe_name, pipeline in pipelines.items():
    pipeline.fit(X_ohe_selected, y.values.ravel())
    scores[(pipe_name,'train')] = accuracy_score(y, pipeline.predict(X_ohe_selected))
    scores[(pipe_name,'test')] = accuracy_score(y_s, pipeline.predict(X_ohe_selected_s))

pd.Series(scores).unstack()

,test,train
gb,0.826518,0.921897
lgb,0.805452,1.000000
mlp,0.795539,0.951185
rf,0.831475,1.000000
tree,0.764560,1.000000


In [ ]:
for pipe_name, pipeline in pipelines.items():
  #results = cross_val_score(pipeline, X_ohe_selected, y, scoring='roc_auc', cv=5)
  #print('cv score('+pipe_name+'):', np.average(results), '+-', np.std(results))
  print(pipe_name)
  predscore = pd.DataFrame(pipeline.predict_proba(X_ohe_selected_s)[:,0:1], columns=['pred_score'])
  ID_s.join(predscore).to_csv('./output/'+  pipe_name + '_' + file_test_name + '_with_pred.csv', index=False)

  imp = pd.DataFrame([pipeline.named_steps['est'].feature_importances_], columns=X_ohe_columns[selector.support_])
  imp.T.to_csv('./output/'+  pipe_name + '_feature_importances.csv', index=True)

tree
rf
gb
lgb


KeyError: ignored